In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('src/taggerSystem/')
from data_util import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
from simpleLSTMWithNNetModel import Model, reloadModel
from trainModel import trainModel
import tensorflow as tf
import os
import pprint
import numpy as np
import pickle
pp = pprint.PrettyPrinter(indent=4)

In [3]:
data_train = "data/icd9NotesDataTable_train.csv"
data_valid = "data/icd9NotesDataTable_valid.csv"
# data_train = "data/smallIcd9NotesDataTable_train.csv"
# data_valid = "data/smallIcd9NotesDataTable_valid.csv"
# vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
# wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
vocab = 'data/icd9Vocab.txt'# vocab for our data
wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
# output_path = 'results/model_description'
# log_output = output_path + "log"
# vocab = 'data/icd9Vocab.txt'# vocab for our data
# wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
# START_TOKEN = "<s>"
# END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 300 # this should not be manually set and should instead come from the word
# vectors. Or maybe it's good that we have to set it so we know for sure the size
# of word vecs we're using
maxAllowedNoteLength = 1000
max_grad_norm = 5
codeIdx = 9
textIdx = 6
learning_rate = 0.001
training_epochs = 100
batch_size = 256
n_input = 1
# n_steps = 10
n_hidden = 200
numNNetLayers = 4
# n_classes = helper.n_labels
output_keep_prob = 0.5
input_keep_prob = 1# not sure why there are two but example used 0.5 for output. Check with TA
numLayers = 1
# embeddingSize = embeddings.shape[1]
# print('Embedding size is %d'%(embeddingSize))


In [4]:
output_path = input(prompt = 'Where should models and performances be saved to?')
output_path = os.path.join('results', output_path)
if output_path == 'results/':
    output_path = 'results/temp'
if not os.path.exists(output_path):
    os.makedirs(output_path)

Where should models and performances be saved to?3LayerNNet_longRun


In [5]:
print(output_path)

results/3LayerNNet_longRun


In [6]:
sizeList = [n_hidden, 150, 75] # these are the weights we wil be using
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx,
    helperLoadPath = output_path)
# helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
#     data_train = data_train, data_valid = data_valid, 
#     maxAllowedNoteLength = maxAllowedNoteLength, 
#     codeIdx = codeIdx, textIdx = textIdx)
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
helper.save(output_path)# token2id and max length saved to output_path
sizeList.append(helper.n_labels)
# sizeList.(numNNetLayers,helper.n_labels)
# np.savetxt(os.path.join(output_path, 'yDev.gz'),yDev)
# #np.save(os.path.join(modelRunOutputPath, 'xDev.npy'), xDev)
# np.savetxt(os.path.join(output_path, 'devTrueIdxs.gz'), lastTrueWordIdx_dev)

total_batches = (xTrain.shape[0]//batch_size)
print('Total number of batches per epoch %d'%(total_batches))
print('Max note length %d'%(helper.max_length))
print('Number of Icd9 codes %d'%(helper.n_labels))
print('There are a total of {} icd9 codes'.format(len(helper.icdDict.keys())))
pp.pprint(helper.icdDict)
print('icd9 present')
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec len = %d'%(embeddings.shape))

/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((39541, 1000), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((39541, 19), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((13181, 1000), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((13181, 19), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


Total number of batches per epoch 154
Max note length 1000
Number of Icd9 codes 19
There are a total of 19 icd9 codes
{   'cat:1': 5,
    'cat:10': 10,
    'cat:11': 3,
    'cat:12': 17,
    'cat:13': 16,
    'cat:14': 13,
    'cat:15': 4,
    'cat:16': 9,
    'cat:17': 8,
    'cat:18': 18,
    'cat:19': 2,
    'cat:2': 15,
    'cat:3': 1,
    'cat:4': 12,
    'cat:5': 7,
    'cat:6': 11,
    'cat:7': 6,
    'cat:8': 0,
    'cat:9': 14}
icd9 present
xDev shape: nObs = 13181, nWords = 1000
yDev shape: nObs = 13181, nClasses = 19
xTrain shape: nObs = 39541, nWords = 1000
yTrain shape: nObs = 39541, nClasses = 19
Embeddings shape: nWords = 10008, wordVec len = 300


In [19]:
numNNetLayers

4

In [20]:
print('Max note length %d'%(helper.max_length))
print('Number of Icd9 codes %d'%(helper.n_labels))
print('There are a total of {} icd9 codes'.format(len(helper.icdDict.keys())))
pp.pprint(helper.icdDict)
print('icd9 present')
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec len = %d'%(embeddings.shape))

Max note length 1000
Number of Icd9 codes 19
There are a total of 19 icd9 codes
{   'cat:1': 5,
    'cat:10': 10,
    'cat:11': 3,
    'cat:12': 17,
    'cat:13': 16,
    'cat:14': 13,
    'cat:15': 4,
    'cat:16': 9,
    'cat:17': 8,
    'cat:18': 18,
    'cat:19': 2,
    'cat:2': 15,
    'cat:3': 1,
    'cat:4': 12,
    'cat:5': 7,
    'cat:6': 11,
    'cat:7': 6,
    'cat:8': 0,
    'cat:9': 14}
icd9 present
xDev shape: nObs = 13181, nWords = 1000
yDev shape: nObs = 13181, nClasses = 19
xTrain shape: nObs = 39541, nWords = 1000
yTrain shape: nObs = 39541, nClasses = 19
Embeddings shape: nWords = 10008, wordVec len = 300


In [21]:
sizeList

[200, 150, 75, 19]

In [22]:
hyperParamDict = {'EMBED_SIZE': EMBED_SIZE,
                  'maxNoteLength': maxAllowedNoteLength,
                  'maxGradNorm': max_grad_norm,
                  'outputKeepProb': output_keep_prob,
                  'inputKeepProb': input_keep_prob,
                  'learningRate': learning_rate,
                  'trainingEpochsMax': training_epochs,
                  'batchSize': batch_size,
                  'n_hidden': n_hidden,
                 'numLayers': numLayers,
                 'sizeList':sizeList}
pp.pprint(hyperParamDict)
with open(os.path.join(output_path, 'hyperParamDict.pickle'), 'wb') as handle:
    pickle.dump(hyperParamDict, handle, protocol = 2)
    # dumping with 2 because ALTUD uses python 2.7 right now.

{   'EMBED_SIZE': 300,
    'batchSize': 256,
    'inputKeepProb': 1,
    'learningRate': 0.001,
    'maxGradNorm': 5,
    'maxNoteLength': 1000,
    'n_hidden': 200,
    'numLayers': 1,
    'outputKeepProb': 0.5,
    'sizeList': [200, 150, 75, 19],
    'trainingEpochsMax': 100}


In [24]:
from trainModel import trainModel
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
trainModel(helperObj = helper, embeddings = embeddings, hyperParamDict = hyperParamDict, 
          xDev = xDev, xTrain = xTrain, yDev = yDev, yTrain = yTrain, 
           lastTrueWordIdx_dev = lastTrueWordIdx_dev, 
           lastTrueWordIdx_train = lastTrueWordIdx_train,
           training_epochs = training_epochs, 
           output_path = output_path, batchSizeTrain = batch_size,
           sizeList = sizeList,
           maxIncreasingLossCount = 100, batchSizeDev = 1500, chatty = True)

shape of embeddings
(?, 1000, 300)
<class 'tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.MultiRNNCell'>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 300)
cell output size
200
cell state size
(LSTMStateTuple(c=200, h=200),)
output shape
(?, 1000, 200)
offset shape
(?, 1)
output shape new shape
(?, 200)
flattened indices shape
(?, 1)
output flattened shape
(?, 200)
(200, 150)
W_1 shape
(150,)
b_1 shape
(150, 75)
W_2 shape
(75,)
bias shape
(19,)
U shape
(75, 19)
bias shape
(19,)
output wx + b
(?, 19)
(?, 19)


/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


***************************
***************************
Running on epoch 0
***************************
***************************
running iteration 0 with loss 0.692090 at time 1.140060
running iteration 25 with loss 0.494451 at time 21.777998
running iteration 50 with loss 0.479277 at time 42.134598
running iteration 75 with loss 0.498533 at time 62.550422
running iteration 100 with loss 0.474625 at time 83.140333
running iteration 125 with loss 0.470814 at time 103.349360
running iteration 150 with loss 0.462965 at time 123.756083
average training loss 0.494690
test loss 0.489485
Total run time was 139.324978
New best model found. Saving
results/3LayerNNet_longRun
***************************
***************************
Running on epoch 1
***************************
***************************
running iteration 0 with loss 0.473876 at time 0.822015
running iteration 25 with loss 0.444436 at time 21.519092
running iteration 50 with loss 0.452394 at time 42.013957
running iteration 75 

In [116]:
output_path

'results/temp'

In [11]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))
with tf.Session() as session:
    tf.global_variables_initializer().run()
    modelDict = reloadModel(session = session,
                            saverCheckPointPath = 'results/temp/',
                            saverMetaPath = 'results/temp/bestModel.meta')
    print('here we go')
    for i in range(3):
        pred_y = session.run(modelDict['y_last'],feed_dict={modelDict['xPlaceHolder']: xDev,
                                      modelDict['trueWordIdxs']:lastTrueWordIdx_dev,
                                      modelDict['outputKeepProb']: 1.0,
                                      modelDict['inputKeepProb']: 1.0}, ) 
        validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                             labels = tf.cast(yDev, tf.float32))
        validLoss = tf.reduce_mean(validLoss)
        validLoss = validLoss.eval()
        print('test loss %f'%(validLoss))
        print('***********************************************')

here we go
test loss 0.597082
***********************************************
test loss 0.597082
***********************************************
test loss 0.597082
***********************************************


In [1]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

In [2]:
xDev.shape

NameError: name 'xDev' is not defined